In [ ]:
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

In [ ]:
!pip install lightning

In [ ]:
!pip install librosa

In [ ]:
%load_ext tensorboard

In [ ]:
logs_base_dir = "logs/acoustic"
ckpt_acouastic = "./checkpoints/am_pitche_stats_with_vocoder.ckpt"
ckpt_vocoder = "./checkpoints/vocoder.ckpt"
ckpt_vocoder

In [ ]:
%tensorboard --logdir {logs_base_dir}

In [ ]:
import os

from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import StochasticWeightAveraging
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.tuner.tuning import Tuner

from training.modules import AcousticModule, VocoderModule

os.environ["CUDA_LAUNCH_BLOCKING"]="1"
CUDA_LAUNCH_BLOCKING=1

In [ ]:
default_root_dir="logs/acoustic"
accelerator="cuda"
ckpt_acoustic="./checkpoints/am_pitche_stats_with_vocoder.ckpt"
ckpt_vocoder="./checkpoints/vocoder.ckpt"
accumulate_grad_batches=5

tensorboard = TensorBoardLogger(save_dir=default_root_dir)

trainer = Trainer(
    logger=tensorboard,
    # Save checkpoints to the `default_root_dir` directory
    default_root_dir=default_root_dir,
    accelerator=accelerator,
    accumulate_grad_batches=accumulate_grad_batches,
    max_epochs=-1,
)

# Load the pretrained weights for the vocoder
vocoder_module = VocoderModule.load_from_checkpoint(
    ckpt_vocoder,
)

module = AcousticModule.load_from_checkpoint(
    ckpt_acoustic,
    vocoder_module=vocoder_module,
)

# vocoder_module = VocoderModule()
# module = AcousticModule()

train_dataloader = module.train_dataloader()

trainer.fit(model=module, train_dataloaders=train_dataloader)
